In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Apr 19 10:39:01 2016

@author: Martijn Sol
"""

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy.linalg import toeplitz
import time

## Calculation of the solution of the Time Dependent Schrodinger Equation


T = 0.5                 # end time
Nt = 501                # Number of timesteps
t = np.linspace(0,T,Nt) # time
dt = T / (Nt - 1)       # size timestep

L = 50                  
Nx = 4001               # Number of spacial points
x = np.linspace(-L,L,Nx)
dx = L / (Nx - 1)       # size space step


def barrier(h,left,right):
    # Creates a potential barrier of heigth h between the left and right 
    # boundaries
    V = h * 0.5 * (np.sign(x-left) - np.sign(x-right))
    return V
    
def H(Nx,dx):
    # Creates the Hamitonian in matrix form
    toe = np.zeros((Nx))
    toe[0] = 2
    toe[1] = -1
    H1 = toeplitz(toe,toe)
    
    # Dirichlet BC
    #H1[0,Nx-1] = 999999
    #H1[Nx-1,0] = 999999
    
    # Periodic BC
    H1[0,Nx-1] = -1
    H1[Nx-1,0] = -1
    
    H = H1/(dx**2) + V_m
    return H


In [ ]:
## Initial conditions
k_f = np.fft.fftfreq(Nx,dx) # array in k space
k_n = np.fft.fftshift(k_f)

width = 1       # spread in k space
k0 = 8         # Group velocity
psi_k0 = np.exp(-(k_f - k0) ** 2 / (2 * width **2)) # Initial condition in k space
#normk = np.linalg.norm(psi_k0) ** 2
#psi_k0 = psi_k0 / normk

psi_x0 = np.fft.fftshift(np.fft.ifft(psi_k0, Nx)) # Initial condition in x space
#normx = np.linalg.norm(psi_x0) ** 2
#psi_x0 = psi_x0 / normx

## Create the step matrix
V = barrier(-3000, 5, 7)     # Set potential
V_m = np.diag(V)
H = H(Nx,dx)
Hplus = np.eye(Nx) + 0.5 * 1j * dt * H
Hmin = np.eye(Nx) - 0.5 * 1j * dt * H
HplusInv = np.linalg.inv(Hplus)
M = np.dot(HplusInv, Hmin)

In [ ]:
## Calculation of the wave function in time
psi_x = np.zeros((Nt,Nx), dtype = complex)
psi_x[0,:] = psi_x0
psi_k = np.zeros((Nt,Nx), dtype = complex)
psi_k[0,:] = psi_k0
for i in range(Nt-1):
    psi_x[i+1,:] = np.dot(M, psi_x[i,:])
    psi_k[i+1,:] = np.fft.fft(psi_x[i+1,:])


In [ ]:
psi_kAbs = np.abs(psi_k) ** 2
R = np.zeros(((Nx-1)/2,1))
T = np.zeros(((Nx-1)/2,1))
for i in range(int((Nx-1)/2)):
    j = i + 1
    # Only calculate coefficients if this k is sufficiently in the initial condition
    if psi_kAbs[0,j] < 1e-20:
        R[i,0] = 0
        T[i,0] = 0
    else:
        R[i,0] = psi_kAbs[Nt-1, Nx - 1 - i] / psi_kAbs[0,j]
        T[i,0] = psi_kAbs[Nt-1,j] / psi_kAbs[0,j]
    

In [ ]:
## Plots of Reflection and Transmission coefficients
fig = plt.figure(figsize=(8,6))
plt.plot(k_f[1:(Nx+1)/2], T, 'b-')
plt.ylabel('Transmission',fontsize=18)
plt.xlabel('k',fontsize=18)
plt.axis([0, 20, 0, 1.5])
plt.grid()
plt.show()

fig = plt.figure(figsize=(8,6))
plt.plot(k_f[1:(Nx+1)/2], R, 'b-')
plt.ylabel('Reflection',fontsize=18)
plt.xlabel('k',fontsize=18)
plt.axis([0, 20, 0, 1.5])
plt.grid()
plt.show()

# The plots show the transmission spectrum that we expect: oscilations with 
# some values of k being fully transmitted. Higher values of k (and thus energy)
# are barely effected by the potential, thus having a transmission of 1.
# Very low and very high k give incorrect transmission values (T>1) . This is 
# due to the finite dx, which causes these values to not be properly represented.


In [ ]:
## Animation in x space
fig, ax = plt.subplots()

line, = ax.plot(x, np.abs(psi_x[0,:])**2)

def animate(i):
    line.set_ydata(np.abs(psi_x[i,:])**2)  # update the data
    return line,

#Init only required for blitting to give a clean slate.
def init():
    line.set_ydata(np.ma.array(x, mask=True))
    return line,

ani = animation.FuncAnimation(fig, animate, np.arange(1, Nt), init_func=init,
    interval=20, blit=True)
plt.show()


In [ ]:
# Animation in k space
fig, ax = plt.subplots()

#x = np.arange(0, L, dx)        # x-array
line, = ax.plot(k_f, np.abs(psi_k[0,:]) ** 2)

def animate(i):
    line.set_ydata(np.abs(psi_k[i,:]) ** 2)  # update the data
    return line,

#Init only required for blitting to give a clean slate.
def init():
    line.set_ydata(np.ma.array(k_f, mask=True))
    return line,

ani = animation.FuncAnimation(fig, animate, np.arange(1, Nt), init_func=init,
    interval=20, blit=True)
plt.show()
